In [24]:
!pip install pdfplumber fitz frontend


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [frontend]


In [11]:
from dataclasses import dataclass
from __future__ import annotations

Creation of the Node class: 
- name of the node
- description of the node
- children of the node

In [12]:
@dataclass
class Node:
    name: str
    desc: str
    children: list[Node] | None = None

    def get_name(self) -> str: 
        return self.name
        
    def get_desc(self) -> str: 
        return self.desc

    def has_children(self) -> bool: 
        return self.children is not None

    def get_children(self) -> list[Node] | None:
        return self.children


We create the graph of the NACE: https://ec.europa.eu/eurostat/documents/3859598/5902521/KS-RA-07-015-EN.PDF.pdf/dd5443f5-b886-40e4-920d-9df03590ff91?t=1414781457000

In [15]:
import requests

def download_pdf(url, filename):
    response = requests.get(url)
    response.raise_for_status()  # Check if download was successful
    
    with open(filename, 'wb') as f:
        f.write(response.content)
    
    print(f"PDF downloaded as: {filename}")
    

In [20]:
url = "https://ec.europa.eu/eurostat/documents/3859598/5902521/KS-RA-07-015-EN.PDF.pdf/dd5443f5-b886-40e4-920d-9df03590ff91?t=1414781457000"
file_path = "code_naf_grpo/NACE.pdf"
download_pdf(url, file_path)

PDF downloaded as: code_naf_grpo/NACE.pdf


Let's have a look of the organisation of the PDF : 

In [25]:
import fitz 

def get_pdf_outline(pdf_path):
    doc = fitz.open(pdf_path)
    outline = doc.get_toc()  # Get Table of Contents
    
    for level, title, page in outline:
        indent = "  " * (level - 1)  # Indent based on hierarchy level
        print(f"{indent}{title} (Page {page})")
    
    doc.close()
    return outline

# Usage
outline = get_pdf_outline(file_path)

RuntimeError: Directory 'static/' does not exist